# Imports

In [2]:
# For JSON imports
import json

#For DF, CSV, Excel
import pandas as pd

# Open Data
Contains the coordinates and metro lines that stop at those coordinates. Could be used in the explanation of nieuwmarkt.

## Data Exploration

In [43]:
#Import JSON file
with open("../Data/Original/TRAMMETRO_PUNTEN_2019.json") as gvb_data:
    stations = json.load(gvb_data)
    stations = stations["features"]
    
    #Example line in file:
    #Lines that stop at the given station
    print("Lines: ", stations[0]["properties"]["Lijn_select"])

    #Coordinates of the station
    print("Coordinates: ", stations[0]["geometry"]["coordinates"])
    
    #Construct actual file

    #Loop over all stations in file
    for station in stations:
        #Select only stations where the tram stops
        if station["properties"]["Modaliteit"] == "Tram":
            
            #Per station, select which lines stop there and the coordinates of the station            
            tram_dict = {"Lines": station["properties"]["Lijn_select"], "Coordinates": station["geometry"]["coordinates"]}
            
            #Append the given results to the file and save it            
            with open("../Data/Modified/TramStations.json", "a") as f:
                json.dump(tram_dict, f)
        
        #Select only stations where the metro stops
        elif station["properties"]["Modaliteit"] == "Metro":
            
            #Per station, select which lines stop there and the coordinates of the station            
            metro_dict = {"Lines": station["properties"]["Lijn_select"], "Coordinates": station["geometry"]["coordinates"]}
            
            #Append the given results to the file and save it
            with open("../Data/Modified/MetroStations.json", "a") as f:
                json.dump(metro_dict, f)

Lines:  02|11|12|13|17
Coordinates:  [4.893349, 52.376064]


# Sample Data
Old version of the data, used to make a start in the data prep

## Data Exploration

### Ritten
Contains the "Ritten" data for each station. Can be used to count the number of people that originated from the station on an hourly basis

#### Import the Dataset
First we'll import the dataset and check what it contains

In [3]:
#Import the full dataset
ritten = pd.read_excel("../../../Data_thesis/Sample_data/GVB/full.xlsx")

In [4]:
#Check contents
ritten.head()

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalRitten
0,2018-12-24,00:00 - 00:59,0,Overig,0.0,0.000000e+00,77.0
1,2018-12-24,00:00 - 00:59,5046,Rembrandtplein,489627975.0,5.236626e+09,13.0
2,2018-12-24,00:00 - 00:59,5096,Centraal Station,490049319.0,5.237968e+09,10.0
3,2018-12-24,00:00 - 00:59,6063,Leidseplein,488064547.0,5.236476e+09,10.0
4,2018-12-24,01:00 - 01:59,0,Overig,0.0,0.000000e+00,132.0


#### Construct Dataframes
Now that the contents is clear, we construct Dataframes for each station that is of interest to us. 

##### Metro
- *Nieuwmarkt*
- *Rokin*

##### Tram
- *Niewezijdse Kolk*
- *Dam*
- *Spui*
- *Rokin*

In [12]:
#List al stations
stations = ["Nieuwmarkt", "Niewezijdse Kolk", "Dam", "Spui"]

#Select all the rows that have the station name in departure station
ritten_df = ritten[ritten["VertrekHalteNaam"].isin(stations)]

In [13]:
#Example
ritten_df.head()

,Datum,UurgroepOmschrijving (van vertrek),VertrekHalteCode,VertrekHalteNaam,VertrekXCoord,VertrekYCoord,AantalRitten
5,2018-12-24,01:00 - 01:59,5031,Dam,489348393.0,5.237341e+09,58.0
8,2018-12-24,01:00 - 01:59,5068,Dam,489124121.0,5.237436e+09,28.0
16,2018-12-24,02:00 - 02:59,5031,Dam,489348393.0,5.237341e+09,103.0
18,2018-12-24,02:00 - 02:59,5068,Dam,489124121.0,5.237436e+09,25.0
26,2018-12-24,03:00 - 03:59,5031,Dam,489348393.0,5.237341e+09,39.0


#### DF to File
Save the dataframe in a file, so that it can be imported for other uses. Later this will be probably be removed as the dataframe can simple function as input to another funtion. 

In [14]:
ritten_df.to_csv("../../../Data_thesis/Sample_data/GVB/ritten.csv")